# PROYECTO V - PROBLEMA DE REGRESIÓN

En este proyecto de problema de regresión hemos decidido usar un [Dataset](https://www.kaggle.com/competitions/playground-series-s4e9/overview) de precio de coches usados. El proyecto está enfocado a las personas que quieran vender su coche y quieran tener una idea del precio de venta de antemano. Para poder hacer un estudio del Dataset y crear un Modelo que prediga los precios de venta del coche lo primero que vamos a hacer es una limpieza del Dataset, ver los valores faltantes o outliers y poder tratarlos.

Tras la limpieza de los datos los datos para calcular el precio del coche son:
- Características: 
    - brand
    - model
    - model_year
    - milage
    - fuel_type
    - engine
    - transmission
    - ext_col
    - int_col
    - accident
    - price
- Etiqueta: precio del coche

Vamos a ver los pasos que hemos realizado para llegar a tener las columnas listas.

## PASO 1: REVISAR EL DATASET

### Importar el dataset.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('data/used_cars.csv')

### Imprimir cabeceras del dataset y tener una idea clara de los tipos de datos que tenemos.

In [6]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


### Eliminar la columna 'clean_title' porque no lo vamos a usar.

In [7]:
df.drop(['clean_title'], axis=1, inplace=True)

### Imprimir los elementos y subniveles que tiene cada columna.

In [8]:
cols_cat = ['brand','model','milage','fuel_type','engine','transmission','ext_col','int_col','accident','price']

for col in cols_cat:
    print(f'Column {col}: {df[col].nunique()} subniveles')

Column brand: 57 subniveles
Column model: 1898 subniveles
Column milage: 2818 subniveles
Column fuel_type: 7 subniveles
Column engine: 1146 subniveles
Column transmission: 62 subniveles
Column ext_col: 319 subniveles
Column int_col: 156 subniveles
Column accident: 2 subniveles
Column price: 1569 subniveles


### Generar estadísticas descriptivas de las columnas numéricas de nuestro dataset.

In [9]:
df.describe()

,model_year
count,4009.000000
mean,2015.515590
std,6.104816
min,1974.000000
25%,2012.000000
50%,2017.000000
75%,2020.000000
max,2024.000000


### Mostrar todas las filas y columnas

In [10]:
# Mostrar todas las filas y columnas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

print(df)

         brand                            model  model_year      milage  \
0         Ford  Utility Police Interceptor Base        2013  51,000 mi.   
1      Hyundai                     Palisade SEL        2021  34,742 mi.   
2        Lexus                    RX 350 RX 350        2022  22,372 mi.   
3     INFINITI                 Q50 Hybrid Sport        2015  88,900 mi.   
4         Audi        Q3 45 S line Premium Plus        2021   9,835 mi.   
...        ...                              ...         ...         ...   
4004   Bentley             Continental GT Speed        2023     714 mi.   
4005      Audi             S4 3.0T Premium Plus        2022  10,900 mi.   
4006   Porsche                           Taycan        2022   2,116 mi.   
4007      Ford                     F-150 Raptor        2020  33,000 mi.   
4008       BMW                     X3 xDrive30i        2020  43,000 mi.   

          fuel_type                                             engine  \
0     E85 Flex Fuel  300.

### Limpieza de la columna 'milage'

Como vemos la columna'milage' está en tipo de dato object y además el formato no nos dejaría hacer un análisis. Por ello, hemos decidido eliminar el string 'mi.', reemplazar el ',' por '.' y cambiarlo a tipo float.

In [11]:
df['milage'] = df['milage'].str.replace('mi.', '', regex=False).str.replace(',', '.').astype(float)

### Limpieza de la columna 'price'

Hacemos lo mismo con la columna 'price', eliminando el signo de '$' y cambiando el ',' por '.'

In [12]:
df['price'] = df['price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

### Limpieza de la columna 'engine'

Esta columna ha sido la más problemática, porque nos dimos cuena que había muchas diferentes opciones y variables. Para esto hemos usado expresiones regulares (regex) para buscar recuencias de carácteres y patrones comunes en esta columna y crear una nueva columna para cada uno de ellos.

In [13]:
df['engine'].unique()

array(['300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability',
       '3.8L V6 24V GDI DOHC', '3.5 Liter DOHC', ...,
       '136.0HP 1.8L 4 Cylinder Engine Gasoline Fuel',
       '270.0HP 2.0L 4 Cylinder Engine Gasoline Fuel',
       '420.0HP 5.9L 12 Cylinder Engine Gasoline Fuel'],
      shape=(1146,), dtype=object)

**Creación de la columna 'engine_size' mediante regex a partir de 'engine'** : Hemos tomado el carácter 'L' como filtro, para extraer la capacidad del motor del coche. 

Por ejemplo lo que antes era:

    ['engine] = 3.8L V6 24V GDI DOHC

Ahora es:

    ['engine_size] 3.8


In [14]:
df['engine_size'] = df['engine'].str.extract(r'(\d.\d+)\s?L').astype(float)

**Creación de la columna 'engine_type' meidante regex a partir de la columna 'engine'**: Hemos tomado como carácteres delimitantes las palabras 'I\d', 'Electric', 'Hybrid', 'Turbo', 'Diesel'.

Por ejemplo lo que antes era:

    ['engine'] = 534.0HP Electric Motor Electric Fuel System

Ahora es:

    [engine_type'] = Electric

Y además, las columnas vacías hemos reestablecido como 'Unknown'


In [15]:
df['engine_type'] = df['engine'].str.extract(r'(V\d|I\d|Electric|Hybrid|Turbo|Diesel)', expand=False)
df['engine_type'] = df['engine_type'].fillna('Unknown')

**Creación de la columna 'engine_hp' mediante regex a partir de la columna 'engine'**: Hemos tomado como carácter delimitador el string 'HP'.

Por ejemplo lo que antes era:

    ['engine'] = 534.0HP Electric Motor Electric Fuel System

Ahora es:

    [engine_hp'] = 534.0


In [16]:
df['engine_hp'] = df['engine'].str.extract(r'(\d+.?\d*)\s?HP')
df['engine_hp'] = pd.to_numeric(df['engine_hp'], errors='coerce')

### Eliminamos las columnas 'engine', 'ext_col' e 'int_color'

In [17]:
df = df.drop(columns=['engine','ext_col','int_col'])

### Normalizar la columna 'accident'

Hemos convertido en valores booleanos la columna 'accident':

    'None reported' => 0
    'At least 1 accident or damage reported' => 1
    'nan' => 0

In [18]:
df['accident'] = df['accident'].map({
    'None reported': 0,
    'At least 1 accident or damage reported': 1
}).where(pd.notna(df['accident'])).astype('Int64')

df['accident'] = df['accident'].fillna(0).astype('Int64')

### Pasando la columna price a int

In [20]:
df["price"] = df["price"].astype(int)

### Suplantando los valores vacios de Fuel_type por Electric (corroborado en la otra columna engine_type)

In [22]:
df["fuel_type"] = df["fuel_type"].fillna("Electric")

### Luego de evaluar decidimos borrar la columna engine_type

In [24]:
df.drop(['engine_type'], axis=1, inplace=True)

## DATASET LIMPIO Y NORMALIZADO 👇

In [25]:
new_df = df.copy()
new_df.to_csv('data/cleaned_dataset.csv', index=False)

new_df.info()
new_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         4009 non-null   object 
 1   model         4009 non-null   object 
 2   model_year    4009 non-null   int64  
 3   milage        4009 non-null   float64
 4   fuel_type     4009 non-null   object 
 5   transmission  4009 non-null   object 
 6   accident      4009 non-null   Int64  
 7   price         4009 non-null   int64  
 8   engine_size   3773 non-null   float64
 9   engine_hp     3201 non-null   float64
dtypes: Int64(1), float64(3), int64(2), object(4)
memory usage: 317.2+ KB


,brand,model,model_year,milage,fuel_type,transmission,accident,price,engine_size,engine_hp
0,Ford,Utility Police Interceptor Base,2013,51.000,E85 Flex Fuel,6-Speed A/T,1,10300,3.7,300.0
1,Hyundai,Palisade SEL,2021,34.742,Gasoline,8-Speed Automatic,1,38005,3.8,NaN
2,Lexus,RX 350 RX 350,2022,22.372,Gasoline,Automatic,0,54598,3.5,NaN
3,INFINITI,Q50 Hybrid Sport,2015,88.900,Hybrid,7-Speed A/T,0,15500,3.5,354.0
4,Audi,Q3 45 S line Premium Plus,2021,9.835,Gasoline,8-Speed Automatic,0,34999,2.0,NaN


## ANÁLISIS EXPLORATORIO DE DATOS